<a href="https://colab.research.google.com/github/ming-x-yue/hackos-4/blob/main/HackBio_Random_Forest_Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the HackBio Random Forest Quickstart

* You can run this notebook without making any changes to the code.

* If this is your first time working with ML for bio, we recommend you start by playing around with different parameter options in the section called **Important! Choose your hyperparameters!** - you can modify the values of the parameters underneath.

* By the end of the notebook, you should be able to see your score (the MRRMSE score).

* Please be sure to download a copy to your computer - if you don't create a copy, you may lose your changes!

* Good luck, and have fun!

### Download Data

In [ ]:
!wget https://github.com/aniketsrinivasan/hackos-4/raw/refs/heads/main/dataset/de_test_split.parquet

--2025-04-02 18:08:58--  https://github.com/aniketsrinivasan/hackos-4/raw/refs/heads/main/dataset/de_test_split.parquet
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aniketsrinivasan/hackos-4/refs/heads/main/dataset/de_test_split.parquet [following]
--2025-04-02 18:08:58--  https://raw.githubusercontent.com/aniketsrinivasan/hackos-4/refs/heads/main/dataset/de_test_split.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30477424 (29M) [application/octet-stream]
Saving to: ‘de_test_split.parquet’

de_test_split.parqu 100%[===================>]  29.07M  66.4MB/s    in 0.4s    

2025-04-02 1

In [ ]:
!wget https://github.com/aniketsrinivasan/hackos-4/raw/refs/heads/main/dataset/de_train_split.parquet

--2025-04-02 18:08:59--  https://github.com/aniketsrinivasan/hackos-4/raw/refs/heads/main/dataset/de_train_split.parquet
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aniketsrinivasan/hackos-4/refs/heads/main/dataset/de_train_split.parquet [following]
--2025-04-02 18:08:59--  https://raw.githubusercontent.com/aniketsrinivasan/hackos-4/refs/heads/main/dataset/de_train_split.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95318099 (91M) [application/octet-stream]
Saving to: ‘de_train_split.parquet’

de_train_split.parq 100%[===================>]  90.90M   153MB/s    in 0.6s    

2025-04-

### Preprocess Data

In [ ]:
import pandas as pd
import os
import tqdm
import numpy as np


# Read the file:
train_dataset_path = "de_train_split.parquet"
train_dataset_df = pd.read_parquet(train_dataset_path)

In [ ]:
train_dataset_df.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


In [ ]:
test_dataset_path = "de_test_split.parquet"
test_dataset_df = pd.read_parquet(test_dataset_path)

In [ ]:
test_dataset_df.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
500,T regulatory cells,Sgc-cbp30,LSM-47437,COc1ccc(CCc2nc3cc(-c4c(C)noc4C)ccc3n2C[C@H](C)...,False,-0.124665,0.081496,0.539741,0.197191,0.181400,...,-0.240047,0.639003,-0.063245,-0.160475,1.434513,0.762768,0.495779,0.053277,-0.326704,-0.473541
501,NK cells,BX 912,LSM-1055,O=C(Nc1cccc(Nc2ncc(Br)c(NCCc3c[nH]cn3)n2)c1)N1...,False,0.416993,1.588819,-0.235642,0.230320,0.239342,...,1.225968,0.209653,-0.009077,-0.027860,0.392218,0.169842,0.696410,0.732927,0.141402,0.406739
502,T cells CD4+,BX 912,LSM-1055,O=C(Nc1cccc(Nc2ncc(Br)c(NCCc3c[nH]cn3)n2)c1)N1...,False,-1.134286,0.065469,0.544500,-0.076772,0.254242,...,-0.708817,-0.091768,0.398702,-0.460452,-0.464118,0.293434,-0.155006,0.247944,-0.645191,-2.374144
503,T cells CD8+,BX 912,LSM-1055,O=C(Nc1cccc(Nc2ncc(Br)c(NCCc3c[nH]cn3)n2)c1)N1...,False,0.062209,0.462238,0.363180,-0.158961,0.274419,...,0.270252,0.697346,-0.422461,0.110836,0.258739,0.284098,0.160138,0.065669,-1.256339,-1.493723
504,T regulatory cells,BX 912,LSM-1055,O=C(Nc1cccc(Nc2ncc(Br)c(NCCc3c[nH]cn3)n2)c1)N1...,False,2.498582,0.970093,3.129195,3.045847,3.615967,...,-0.253227,2.018453,0.979286,1.707055,2.291522,1.965766,1.423388,0.683576,-0.539229,0.566867


In [ ]:
CELL_TYPE_NAMES = train_dataset_df['cell_type'].unique()

In [ ]:
DRUG_NAMES = train_dataset_df['sm_name'].unique()

In [ ]:
X_data_train_df = train_dataset_df.iloc[:,0:2]
X_data_train_df.head()

,cell_type,sm_name
0,NK cells,Clotrimazole
1,T cells CD4+,Clotrimazole
2,T cells CD8+,Clotrimazole
3,T regulatory cells,Clotrimazole
4,NK cells,Mometasone Furoate


In [ ]:
X_data_test_df = test_dataset_df.iloc[:,0:2]
X_data_test_df.head()

,cell_type,sm_name
500,T regulatory cells,Sgc-cbp30
501,NK cells,BX 912
502,T cells CD4+,BX 912
503,T cells CD8+,BX 912
504,T regulatory cells,BX 912


In [ ]:
y_data_train_df = train_dataset_df.iloc[:,5:]
y_data_train_df.head()

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0.104720,-0.077524,-1.625596,-0.144545,0.143555,0.073229,-0.016823,0.101717,-0.005153,1.043629,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,0.915953,-0.884380,0.371834,-0.081677,-0.498266,0.203559,0.604656,0.498592,-0.317184,0.375550,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,-0.387721,-0.305378,0.567777,0.303895,-0.022653,-0.480681,0.467144,-0.293205,-0.005098,0.214918,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,0.232893,0.129029,0.336897,0.486946,0.767661,0.718590,-0.162145,0.157206,-3.654218,-0.212402,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,4.290652,-0.063864,-0.017443,-0.541154,0.570982,2.022829,0.600011,1.231275,0.236739,0.338703,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


In [ ]:
y_data_test_df = test_dataset_df.iloc[:,5:]
y_data_test_df.head()

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
500,-0.124665,0.081496,0.539741,0.197191,0.181400,0.176225,-0.152762,0.339240,0.368025,0.018033,...,-0.240047,0.639003,-0.063245,-0.160475,1.434513,0.762768,0.495779,0.053277,-0.326704,-0.473541
501,0.416993,1.588819,-0.235642,0.230320,0.239342,0.756034,0.856384,-0.854756,-0.249477,0.077039,...,1.225968,0.209653,-0.009077,-0.027860,0.392218,0.169842,0.696410,0.732927,0.141402,0.406739
502,-1.134286,0.065469,0.544500,-0.076772,0.254242,0.778740,-1.176965,0.163836,0.680554,0.171205,...,-0.708817,-0.091768,0.398702,-0.460452,-0.464118,0.293434,-0.155006,0.247944,-0.645191,-2.374144
503,0.062209,0.462238,0.363180,-0.158961,0.274419,-0.045575,0.223334,-0.401203,-1.160553,0.045423,...,0.270252,0.697346,-0.422461,0.110836,0.258739,0.284098,0.160138,0.065669,-1.256339,-1.493723
504,2.498582,0.970093,3.129195,3.045847,3.615967,3.493357,0.361326,0.938023,0.529716,1.132383,...,-0.253227,2.018453,0.979286,1.707055,2.291522,1.965766,1.423388,0.683576,-0.539229,0.566867


In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

def encoder(X):
    unique_list_train_cell_type = train_dataset_df["cell_type"].unique()
    unique_list_train_drug_names = train_dataset_df["sm_name"].unique()

    cell_arr = np.zeros(len(unique_list_train_cell_type)+1)
    drug_arr = np.zeros(len(unique_list_train_drug_names)+1)

    try:
        CELL_TYPE_IDX = np.where(unique_list_train_cell_type == X[0])[0][0]
    except IndexError:
        CELL_TYPE_IDX = len(unique_list_train_cell_type)

    try:
        DRUG_NAME_IDX = np.where(unique_list_train_drug_names == X[1])[0][0]
    except IndexError:
        DRUG_NAME_IDX = len(unique_list_train_drug_names)

    cell_arr[CELL_TYPE_IDX] = 1
    drug_arr[DRUG_NAME_IDX] = 1

    return np.concatenate((cell_arr, drug_arr))

encoder(['T cells CD4+','PRT-062607'])

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

### Important! Choose your hyperparameters!

For the following variables:

* `MAX_DEPTH`: the number of search layers, use any number 1 or higher!
* `N_ESTIMATORS`: the number of estimators, use any number 1 or higher
* `CRITERION`: the metric to evaluate error: choose any of the following: {“squared_error”, “absolute_error”, “friedman_mse”, “poisson”}

In [ ]:
MAX_DEPTH=20
N_ESTIMATORS=50
CRITERION="squared_error"

### Train Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


X_train = []
for x_item in tqdm.tqdm(X_data_train_df.values):
    X_train.append(encoder(x_item))

X_test = []
for x_item in tqdm.tqdm(X_data_test_df.values):
    X_test.append(encoder(x_item))


X_train = np.asarray(X_train)
y_train = y_data_train_df.values

clf = RandomForestRegressor(max_depth=MAX_DEPTH, n_estimators=N_ESTIMATORS, criterion=CRITERION)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

100%|██████████| 114/114 [00:00<00:00, 7390.16it/s]


### Calculate Test Accuracy

This is your official score, you want this to be as low as possible

In [ ]:
def get_test_MRRMSE(y_pred, test_dataset_df):
    MRRMSE = 0.0
    N = test_dataset_df.shape[0]
    if len(y_pred) != test_dataset_df.shape[0]:
        raise IndexError("y_pred must have the same number of rows as the test dataset")
    for idx in tqdm.trange(N):
        # print(test_dataset_df.iloc[idx,5:].shape)
        elementwise_diff = y_pred[idx]-test_dataset_df.iloc[idx,5:].values
        elementwise_square = np.power(elementwise_diff, 2)
        mean_row_error = np.mean(elementwise_square)
        MRRMSE += np.sqrt(mean_row_error)/N
        # print()
        # break
    return MRRMSE

MRRMSE = get_test_MRRMSE(y_pred, test_dataset_df)
print()
print(f"Your MRRMSE score, you want this to be as low as possible: {round(MRRMSE,5)}")

100%|██████████| 114/114 [00:02<00:00, 48.58it/s]


Your MRRMSE score, you want this to be as low as possible: 1.18949


### Sanity Check

The following is the model just predicting zeros - ideally, the value you predicted above should be less than the following value.

In [ ]:
MRRMSE = get_test_MRRMSE(np.zeros((114,18211)), test_dataset_df)
print()
print(MRRMSE)

100%|██████████| 114/114 [00:01<00:00, 72.21it/s]


1.2163896383797677
